In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/PatientCategorical.csv')
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       224 non-null    int64  
 1   Gender                    224 non-null    int64  
 2   BMI, kg/m2                224 non-null    int64  
 3   Height/m                  224 non-null    int64  
 4   History of Heart Failure  224 non-null    int64  
 5   working place             224 non-null    int64  
 6   mMRC                      224 non-null    int64  
 7   status of smoking         224 non-null    int64  
 8   Pack History              224 non-null    int64  
 9   Vaccination               224 non-null    int64  
 10  Depression                224 non-null    int64  
 11  Dependent                 224 non-null    int64  
 12  Temperature               224 non-null    int64  
 13  Respiratory Rate          224 non-null    int64  
 14  Heart Rate

In [3]:
print(data.corr())

                               Age    Gender  BMI, kg/m2  Height/m  \
Age                       1.000000 -0.043159    0.192151       NaN   
Gender                   -0.043159  1.000000   -0.004349       NaN   
BMI, kg/m2                0.192151 -0.004349    1.000000       NaN   
Height/m                       NaN       NaN         NaN       NaN   
History of Heart Failure  0.188975  0.005499    0.155071       NaN   
working place             0.054149 -0.143508   -0.053484       NaN   
mMRC                      0.293394  0.097915    0.219995       NaN   
status of smoking        -0.028212 -0.117860   -0.148980       NaN   
Pack History              0.071030  0.063500    0.203391       NaN   
Vaccination               0.450651 -0.049003    0.211173       NaN   
Depression                0.017012 -0.057900   -0.119691       NaN   
Dependent                 0.607915  0.021822    0.149464       NaN   
Temperature              -0.343559 -0.116649   -0.200122       NaN   
Respiratory Rate    

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
y = data["Sputum"]
X = data.drop("Sputum", axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe = Pipeline([('std_scl',StandardScaler())])
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)
feature_names = X.columns
X_train = pd.DataFrame(X_train,columns=feature_names)
X_test = pd.DataFrame(X_test,columns=feature_names)
y_test = y_test.to_frame(name="target")
y_train = y_train.to_frame(name="target")


In [16]:
from sklearn.ensemble import BaggingRegressor

bag_reg = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=3,
                                                      max_features='sqrt',
                                                      splitter='random'),
                            n_estimators=30,
                            max_samples=0.8,
                            max_features=0.9,
                            bootstrap=True,
                            bootstrap_features=False,
                            oob_score=False,
                            warm_start=False,
                            n_jobs=2,
                            random_state=0,
                            verbose=0
                            )

bag_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=3,
                                                      max_features='sqrt',
                                                      splitter='random'),
                 max_features=0.9, max_samples=0.8, n_estimators=30, n_jobs=2,
                 random_state=0)

In [17]:
print(len(bag_reg.estimators_))
print(len(bag_reg.estimators_samples_))
print(len(bag_reg.estimators_features_))

30
30
30


In [18]:
scores = []
for est,features in zip(bag_reg.estimators_,bag_reg.estimators_features_):
    scores.append(est.score(X_test.values[:, features], y_test))
print('Avg. estimator performance:',np.mean(scores))
print('Estimaor performance std. dev.:',np.std(scores))
bag_reg.score(X_test, y_test)

Avg. estimator performance: -0.023863586792947606
Estimaor performance std. dev.: 0.15625352766570047


0.14395641047547625